In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.float_format', lambda v: f'{v:,.2f}')

# To categorize the calendar events into buckets for simplification,
# given there are thousands of unique titles
def categorize_title(title):
    title = str(title).lower()
    if any(keyword in title for keyword in ['home', 'construction', 'building', 'housing']):
        return 'Housing & Construction'
    elif any(keyword in title for keyword in ['manufacturing', 'factory', 'durable goods', 'orders', 'vehicle', 'production']):
        return 'Manufacturing & Industry'
    elif any(keyword in title for keyword in ['employment', 'jobless', 'nonfarm', 'earnings', 'labor', 'income']):
        return 'Labor Market'
    elif any(keyword in title for keyword in ['consumer', 'retail', 'sentiment', 'credit', 'redbook']):
        return 'Consumer & Retail'
    elif any(keyword in title for keyword in ['trade balance', 'current account', 'exports', 'imports', 'export', 'import']):
        return 'Trade & Current Account'
    elif any(keyword in title for keyword in ['oil', 'gasoline', 'natural gas', 'eia', 'inventories']):
        return 'Energy & Commodities'
    elif any(keyword in title for keyword in ['fed', 'interest rate', 'FOMC']):
        return 'Monetary Policy'
    elif any(keyword in title for keyword in ['cpi', 'ppi', 'inflation', 'prices', 'pce', 'gdp', 'spending']):
        return 'Inflation & Prices'
    else:
        return 'Other'

# To parse the raw data into numeric values for analysis
def parse_values(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace(',', '')

    if s.endswith('%'):
        return pd.to_numeric(s[:-1], errors='coerce')

    mult = {'K': 1_000, 'M': 1_000_000, 'B': 1_000_000_000}
    if s and s[-1] in mult:
        num = pd.to_numeric(s[:-1], errors='coerce')
        return num * mult[s[-1]]
    return pd.to_numeric(s, errors='coerce')

In [5]:
# Import data obtained through web scraping application
df_2015 = pd.read_csv(r"Data\raw_calendar_data\2015.csv")
df_2016 = pd.read_csv(r"Data\raw_calendar_data\2016.csv")
df_2017 = pd.read_csv(r"Data\raw_calendar_data\2017.csv")
df_2018 = pd.read_csv(r"Data\raw_calendar_data\2018.csv")
df_2019 = pd.read_csv(r"Data\raw_calendar_data\2019.csv")
df_2020 = pd.read_csv(r"Data\raw_calendar_data\2020.csv")
df_2021 = pd.read_csv(r"Data\raw_calendar_data\2021.csv")
df_2022 = pd.read_csv(r"Data\raw_calendar_data\2022.csv")
df_2023 = pd.read_csv(r"Data\raw_calendar_data\2023.csv")
df_2024 = pd.read_csv(r"Data\raw_calendar_data\2024.csv")
df_2025 = pd.read_csv(r"Data\raw_calendar_data\2025.csv")

In [6]:
df_list = [df_2015, df_2016, df_2017, df_2018, df_2019, df_2020,
           df_2021, df_2022, df_2023, df_2024, df_2025]

calendar_data = pd.concat(df_list, ignore_index = True)
calendar_data = calendar_data.drop(columns = 'Web')

# Deal with NaN values in original 'Day' column
calendar_data['Day'] = calendar_data['Day'].ffill()

# Turn 'Holiday' column into dummy variable
calendar_data['Holiday'] = calendar_data['Holiday'].map({'Holiday': 1}).fillna(0).astype(int)

# Remove all rows that contain no data (doesn't represent an event)
calendar_data = calendar_data.drop(
    calendar_data[(calendar_data['Time'].isna()) & (calendar_data['Currency'].isna())].index)

# Drop all events that do not include a forecast, keep holidays
calendar_data = calendar_data[calendar_data['Forecast'].notna() | (calendar_data['Holiday'] == 1)]

# Make 'Day' column values DateTime
calendar_data['Day'] = pd.to_datetime(calendar_data['Day'])

# Apply the categorization
calendar_data['Category'] = calendar_data['Title'].apply(categorize_title)
calendar_data = calendar_data[['Day', 'Holiday', 'Title', 'Category', 'Actual', 'Forecast']]

# Convert 'Actual' and 'Forecast' columns to numeric
calendar_data['Forecast'] = calendar_data['Forecast'].apply(parse_values)
calendar_data['Actual']   = calendar_data['Actual'].apply(parse_values)

# Add a 'Surprise' column
# Represents the relative surprise above/below forecast
calendar_data['Surprise'] = calendar_data['Actual'] - calendar_data['Forecast']
calendar_data['Surprise'] = calendar_data['Surprise'] / calendar_data['Forecast'].abs()

calendar_data = calendar_data.reset_index(drop = True)

In [7]:
calendar_data.tail()

,Day,Holiday,Title,Category,Actual,Forecast,Surprise
15564,2025-09-05,0,Manufacturing Payrolls (Aug),Manufacturing & Industry,"-12,000.00","-5,000.00",-1.40
15565,2025-09-05,0,Nonfarm Payrolls (Aug),Labor Market,"22,000.00","75,000.00",-0.71
15566,2025-09-05,0,Private Nonfarm Payrolls (Aug),Labor Market,"38,000.00","75,000.00",-0.49
15567,2025-09-05,0,Unemployment Rate (Aug),Labor Market,4.30,4.30,0.00
15568,2025-09-05,0,U.S. Baker Hughes Oil Rig Count,Energy & Commodities,414.00,411.00,0.01


In [8]:
# To fix the issue of multiple events (rows) per day, we take the SUM of surprises for all categories
# everyday, and replace the categorical variable 'Category' for 9 individual variables (one for each
# category) 

holiday_dates = (calendar_data.groupby('Day')['Holiday'].max().reset_index())
calendar_df = (calendar_data.groupby(['Day', 'Category'])['Surprise'].sum().unstack(fill_value=0).reset_index())
calendar_df = calendar_df.merge(holiday_dates, on='Day', how='left')

# Ensure that all weekdays are present in the dataset, even days with no event
all_weekdays = pd.bdate_range(start="2015-01-01", end="2025-09-05")
calendar_df = (calendar_df.set_index('Day').reindex(all_weekdays).fillna(0).rename_axis('Day').reset_index())

if 'Holiday' in calendar_df.columns:
    calendar_df['Holiday'] = calendar_df['Holiday'].astype(int)

num_cols = calendar_df.columns.difference(['Day'])
calendar_df[num_cols] = calendar_df[num_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

In [9]:
#calendar_df.to_csv('calendar_data.csv', index = False)
calendar_df

,Day,Consumer & Retail,Energy & Commodities,Housing & Construction,Inflation & Prices,Labor Market,Manufacturing & Industry,Monetary Policy,Other,Trade & Current Account,Holiday
0,2015-01-01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
1,2015-01-02,0.00,0.00,-2.00,0.00,0.00,-0.11,0.00,0.00,0.00,0
2,2015-01-05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0
3,2015-01-06,0.00,0.00,0.00,0.00,0.00,-0.53,0.00,0.00,0.00,0
4,2015-01-07,0.00,1.95,0.00,0.00,0.07,0.00,0.00,0.00,0.07,0
...,...,...,...,...,...,...,...,...,...,...,...
2782,2025-09-01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
2783,2025-09-02,0.00,0.00,0.00,0.00,0.00,0.01,-0.14,-0.06,0.00,0
2784,2025-09-03,0.00,1.18,0.00,0.00,0.00,0.00,0.00,-0.03,0.00,0
2785,2025-09-04,0.00,6.03,0.00,0.00,-0.23,0.01,0.00,-0.03,-0.01,0
